In [1]:
import numpy as np
from scipy import stats
import pandas as pd
import pickle
import pydot
import random
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
import multiprocessing
import time

import warnings
warnings.filterwarnings('ignore')

# Functions to build tree

In [2]:
def causal_train_test_split(data, predictors, response, treatment, test_size = 0.25, estimation_size = 0.33):
    
    global PROP
    PROP = 1 - estimation_size
    
    train_set, test_set = train_test_split(data, test_size=test_size)
    training_sample, estimation_sample = train_test_split(train_set, test_size=estimation_size)
    training_sample.insert(loc = 0, column = 'TRAIN_ESTIMATION_IND', value = np.ones(len(training_sample)))
    estimation_sample.insert(loc = 0, column = 'TRAIN_ESTIMATION_IND', value = np.zeros(len(estimation_sample)))
    new_train_set = pd.concat([training_sample, estimation_sample])
    new_train_set = new_train_set[['TRAIN_ESTIMATION_IND'] + predictors + treatment + response]
    test_set = test_set[predictors + treatment + response]
    return new_train_set, test_set

In [3]:
def data_split(index, value, dataset):
    """ 
    A function seperate a dataset into two numpy matrices 
    given the index of an attribute and a split value for that attribute
    
    Input:
    ------
        index(int): the index of the columns of the dataset
        value(float): the value to be compared with
        dataset(numpy array): the dataset to split
    
    Output:
    ------:
        left(numpy array): the dataset that is split(left half)
        right(numpy array): the dataset that is split(right half)
    
    """
    dim = dataset.shape[1]
    left, right = np.empty(shape=[0, dim]), np.empty(shape=[0, dim])
    for row in dataset:
        if row[index] < value:
            left = np.append(left, [row], axis = 0)
        else:
            right = np.append(right, [row], axis = 0)
    return left, right

In [4]:
def get_emse(train, est, row, index):
    
    # check the cardinality of the training and estimation samples, if size < *threshold*
    # then can not do the calculation
    train_size = len(train)
    est_size = len(est)
    
    # split both training sample and estimation sample using the same rule
    left_train, right_train = data_split(index, row[index], train)
    left_est, right_est = data_split(index, row[index], est)
    

    ### Calculate the estimated treatment effect
            
    # get the cardinality of training sample for both leaves
    left_train_size = len(left_train)
    right_train_size = len(right_train)
    
    # calculate the treatment effect for both leaves, 
    left_est_response_trt = get_response(left_est, 'treatment') 
    left_est_response_ctl = get_response(left_est, 'control') 
    right_est_response_trt = get_response(right_est, 'treatment') 
    right_est_response_ctl = get_response(right_est, 'control') 
    #check cardinality of each leaf, make sure each leaf has at least *threshold* (chould be changed by user)
    # treatment and n control to do the calculation
    
    left_trt_effect = left_est_response_trt.mean() - left_est_response_ctl.mean()
    right_trt_effect = right_est_response_trt.mean() - right_est_response_ctl.mean()
    
    # then calculated the estimated squared treatment effect
    e_trt_effect = (left_train_size * (left_trt_effect ** 2) + right_train_size * (right_trt_effect ** 2))/(train_size)
    
            
    ### Calculate the estimated variance
    left_var = np.var(left_est_response_trt) / PROP + np.var(left_est_response_ctl) / (1 - PROP)
    right_var = np.var(right_est_response_trt) / PROP + np.var(right_est_response_ctl) / (1 - PROP)
    e_var = (1 / train_size + 1 / est_size) * (left_var + right_var)
    
    
    ### Calculate EMSE
    emse = e_trt_effect - e_var    
    
    return emse
    
def get_split_emse(dataset):
 
    train = dataset[dataset[:,0] == 1]
    est = dataset[dataset[:,0] == 0]

    # initialize values to return
    b_index, b_value, b_score, b_groups = float('inf'), float('inf'), float('-inf'), None
    
    #get the new set of predictors according to PRED_WEIGHT
    pred_lst = sample_by_weight(PRED_WEIGHT, COLUMN_NAMES)
    pred_lst_unique = list(set(pred_lst))
    pred_index_lst = [COLUMN_NAMES.index(x) for x in pred_lst_unique]
    
    
    for index in pred_index_lst:
        for row in train:
            groups = data_split(index, row[index], dataset)
            emse = get_emse(train, est, row, index)
            # if mse score gets improved (reduced actually), update the information
            if emse > b_score:# and emse is not np.nan:
                b_index, b_value, b_score, b_groups = index, row[index], emse, groups   
                
    ret_dict =  {'index':b_index, 'value':b_value, 'groups':b_groups}
    return ret_dict

In [5]:
def get_response(dataset, trt):
    if trt == 'treatment':
        return dataset[dataset[:,-2] == 1][:,-1]
    elif trt == 'control':
        return dataset[dataset[:,-2] == 0][:,-1]

In [6]:
# get the split based on criterion
def get_split(dataset, criterion):
    """ 
    A function to split the data based on splitting criterion specified by user
    
    Input:
    ------
        dataset(np array): a dataset in the form of a numpy matrix
        criterion(str): a str to indicate the criterion specified by user
    
    Output:
    ------:
        the same output as functions get_split_xxx
    
    """    
    if criterion == 'mse':
        return get_split_mse(dataset)
    if criterion == 'causal':
        return get_split_emse(dataset)    
    elif criterion == 'gini':
        return get_split_gini(dataset)

In [7]:
# Create a terminal node value
def to_terminal_gini(group):
    response = group[:,-1]
    return stats.mode(response)[0][0] # this could be optimized

def to_terminal_mse(group):
    response = group[:,-1]
    return np.mean(response)

def to_terminal_emse(group):
    est_trt = get_response(group, 'treatment')
    est_ctl = get_response(group, 'control')
    
    causal_effect = np.mean(est_trt) - np.mean(est_ctl)
    proportion_of_data = (len(est_trt) + len(est_ctl)) / TOTAL_DATA_COUNT
    
    return causal_effect, round(proportion_of_data * 100, 1)
    
    
def to_terminal(group, criterion):
    if criterion == 'gini':
        return to_terminal_gini(group)
    elif criterion == 'mse':
        return to_terminal_mse(group)
    elif criterion == 'causal':
        return to_terminal_emse(group)

In [8]:
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth, criterion):
    
    left, right = node['groups']
    
    left_train = left[left[:,0] == 1]
    left_est = left[left[:,0] == 0]
    right_train = right[right[:,0] == 1]
    right_est = right[right[:,0] == 0]    
    
    left_train_response_trt = get_response(left_train, 'treatment')
    left_train_response_ctl = get_response(left_train, 'control')
    right_train_response_trt = get_response(right_train, 'treatment')
    right_train_response_ctl = get_response(right_train, 'control')  
    
    left_est_response_trt = get_response(left_est, 'treatment')
    left_est_response_ctl = get_response(left_est, 'control')
    right_est_response_trt = get_response(right_est, 'treatment')
    right_est_response_ctl = get_response(right_est, 'control')
    
    del(node['groups'])
    
    if len(left) == 0 or len(right) == 0:
        node['left'] = node['right'] = to_terminal(np.append(left, right, axis = 0), criterion)
        return
    
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left, criterion), to_terminal(right, criterion)
        return
    
    # process left child
    if (len(left) <= min_size or len(left_est_response_trt) <= min_size or len(left_est_response_ctl) <= min_size or
        len(right_est_response_trt) <= min_size or len(right_est_response_ctl) <= min_size or 
        len(left_train_response_trt) <= min_size or len(left_train_response_ctl) <= min_size or 
        len(right_train_response_trt) <= min_size or len(right_train_response_ctl) <= min_size):
        node['left'] = to_terminal(left, criterion)
    else:
        node['left'] = get_split(left, criterion)
        if node['left']['groups'] is None:
            node['left'] = to_terminal(left, criterion)
        else:
            split(node['left'], max_depth, min_size, depth+1, criterion)
        
    # process right child
    if (len(right) <= min_size or len(left_est_response_trt) <= min_size or len(left_est_response_ctl) <= min_size or
        len(right_est_response_trt) <= min_size or len(right_est_response_ctl) <= min_size or 
        len(left_train_response_trt) <= min_size or len(left_train_response_ctl) <= min_size or 
        len(right_train_response_trt) <= min_size or len(right_train_response_ctl) <= min_size):
        node['right'] = to_terminal(right, criterion)
    else:
        node['right'] = get_split(right, criterion)
        if node['right']['groups'] is None:
            node['right'] = to_terminal(right, criterion)
        else:
            split(node['right'], max_depth, min_size, depth+1, criterion)

In [9]:
# Build a decision tree
def causalTree(train, max_depth, min_size, criterion):
    
    global TOTAL_DATA_COUNT, COLUMN_NAMES
    TOTAL_DATA_COUNT = len(train)
    COLUMN_NAMES = list(train.columns[1:-2])
    
    train = np.array(train)
    
#     if criterion == 'mse' or criterion == 'gini':
#         root = get_split(train, criterion)
#         #print(root)
#         split(root, max_depth, min_size, 1, criterion)
        
    if criterion == 'causal':
        root = get_split(train, criterion)
        split(root, max_depth, min_size, 1, criterion)
    return root

# Functions to use causal tree

In [10]:
#print out the tree structure
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[%s < %.3f]' % ((depth * ' ', ([node['index'] - 1]), node['value'])))
        print_tree(node['left'], depth + 1)
        print_tree(node['right'], depth + 1)
    else:
        print('%s[%s, %s%%]' % ((depth * ' ', node[0], node[1])))

        
#causal effect prediction
def causalPredict_helper(node,row):
    if row[node['index'] - 1] < node['value']:
        if isinstance(node['left'], dict):
            return causalPredict_helper(node['left'], row)
        else:
            return node['left'][0]
    else:
        if isinstance(node['right'], dict):
            return causalPredict_helper(node['right'], row)
        else:
            return node['right'][0]    
            
def causalPredict(test, tree):
    #get the information of the trainning set and initialize an empty return dataframe
    column_names = list(test.columns) + ['pred_causal_effect']
    test_matrix = np.array(test)
    ret_matrix = np.empty([0, test_matrix.shape[1] + 1])
    
    #predict for each row
    for row in test_matrix:
        row = np.insert(row, len(row), causalPredict_helper(tree, row))
        ret_matrix = np.append(ret_matrix, [row], axis = 0) 
    
    #return a new dataframe with the predicted value at the end of each row
    ret_df = pd.DataFrame(ret_matrix, columns = column_names)

    return ret_df

# random forest

In [11]:
def rf_sample(train_set, sample_ratio):
    """
    a helper function to create a new random forest training set
    """
    #get a subset of rows based on given sample ratio with replacement/bootsrap
    n_row = int(sample_ratio * len(train_set))
    new_train_subset = train_set.sample(n = n_row, replace = True, axis = 0)
    
    return new_train_subset

In [12]:
# function to build a random forest predictor
def causalRandomForestWeighted(train_set, max_depth, min_size, criterion, sample_ratio, n_trees):
    tree_lst = []
    for i in range(n_trees):
        #get rf sub training data
        rf_train_set = rf_sample(train_set, sample_ratio)
        #build the tree
        tree = causalTree(rf_train_set, max_depth, min_size, criterion)    
        tree_lst.append(tree)
        
    return tree_lst

In [13]:
def predictTestSet(rf, test_set):
       
    n_trees = len(rf)
    n_test_set = len(test_set)
    pred_value_list = np.zeros([1,n_test_set])
    p_value_list = np.zeros([1,n_test_set])  
    
    for tree in rf:
        # get the prediction for each tree
        pred_df = causalPredict(test_set, tree)
        pred_values = np.array(pred_df['pred_causal_effect'])
        # aggregate the prediction
        pred_value_list = pred_value_list + pred_values
        
        # get the number of negative causal effect in the prediction
        pred_values[pred_values >= 0] = 0 # set positive causal effect to 0
        pred_values[pred_values < 0] = 1 # set negative causal effect to 1
        p_value_list = p_value_list + pred_values
        
    #calculate the prediction of bagged trees for each data point
    pred_value_rf = pred_value_list / n_trees #np.round_(pred_value_list / n_trees, decimals = 3)
    #calculate the p value of bagged trees for each data point
    p_value_rf = p_value_list / n_trees #np.round_(p_value_list / n_trees, decimals = 3)
    
    #append the prediction and p_value to the dataset
    ret_data = test_set.copy()
    ret_data['rf_pred_causal_effect'] = pred_value_rf[0]
    # calculate the p value against the hypothesis that the causal effect is not 0
    ret_data['rf_p_value'] = p_value_rf[0]
    ret_data['rf_p_value'] = ret_data['rf_p_value'].apply(lambda row: 2*row if row < 0.5 else 2*(1-row))
    
    return ret_data        

In [14]:
def predictDataPoint(rf, data):
    test_set = pd.DataFrame([data], columns = data.keys())
    predict_list = []
    for tree in rf:
        pred_df = causalPredict(test_set, tree)
        pred_value = pred_df.iloc[0,-1]
        predict_list.append(pred_value)
    # calculate the predicted value    
    rf_pred_value = round(np.mean(predict_list),3)
    # calculate p-value
    positive_ratio = sum(x >0 for x in predict_list)/len(predict_list)
    if positive_ratio <= 0.5:
        p_value = 2*positive_ratio
    else:
        p_value = 2*(1-positive_ratio)
    print("Predicted causal effect: " + str(rf_pred_value) + "\n")
    print("P-value: " + str(p_value))

# weighted random forest

In [15]:
def get_mse(df, pred_col, true_col):
    """
    compute the mse given a dataframe and its corresponding predicted column name and true column name
    """
    mse = np.mean((df[pred_col] - df[true_col]) ** 2)
    return mse

In [16]:
def sample_by_weight(p_weight, p_str):
    """
    sample a random n predictors with replacement by the weight of each predictors
    """
    if len(p_weight) != len(p_str):
        raise ValueError('The length of the predictor list and the weight list must be the same')
    
    #get the probability list of each predictors
    prob_lst = [x/sum(p_weight) for x in p_weight]
    
    #randomly draw a sample (indices) according to the probability distribution
    index_lst = []
    for i in range(len(prob_lst)):
        index_lst.append(np.random.choice(np.arange(0, len(prob_lst)), p = prob_lst))
        
    #get the predictors names from the indices
    sample_pred_lst = [p_str[i] for i in index_lst]
    
    return sample_pred_lst

In [18]:
def shuffle_column(df, col_name):
    """
    shuffle a column of a dataframe
    """
    ret_df = df.copy()
    shuffle_lst = np.array(ret_df[col_name])
    np.random.shuffle(shuffle_lst)
    ret_df[col_name] = shuffle_lst
    return ret_df

In [37]:
data1.head()

,Unnamed: 0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,treatment,error,true_val
0,0,1.571753,-0.348377,1.591885,-0.421881,-0.623928,1.967688,0.541506,-0.118146,2.823623,0.253072,0.0,-0.007519,0.785877
1,1,-1.710885,-0.853989,1.080298,0.686010,0.045241,2.196342,-0.128070,-1.004549,0.955377,1.731303,0.0,-0.000603,-0.855443
2,2,-2.185188,0.673428,0.935457,-0.102679,1.147379,-1.310356,0.520439,0.259379,0.507608,3.954316,0.0,-0.002132,-1.092594
3,3,0.410518,-1.022942,0.968002,2.305879,0.824120,3.345610,1.631188,-0.736464,2.196028,1.552748,0.0,0.001370,0.205259
4,4,1.755752,0.117014,1.223863,0.042935,-0.101483,-0.643004,1.368282,-1.804749,1.627001,1.371938,0.0,-0.003179,0.877876


In [51]:
def get_weighted_rf(train_set, test_set, n_iter, true_col_name, max_depth, min_size, criterion, sample_ratio, n_trees):
    """
    rf: a trained random forest
    test_set: test dataset
    n_iter: number of iterations
    p_str: a list of predictor names
    true_col_name(str): column name of the true value
    
    A function to iteratively get weight on each predictors
    """
    
    #initiate a weight list(equal weight for each predictor to start with)
    global PRED_WEIGHT
    PRED_WEIGHT = [1/len(p_str)] * len(p_str)
    
    for j in range(n_iter):
        #print(j)
        #build the initiate random forest and compute full mse
        rf = causalRandomForestWeighted(train_set, max_depth, min_size, criterion, sample_ratio, n_trees)
        pred_df = predictTestSet(rf, test_set)
        full_mse = get_mse(pred_df, 'rf_pred_causal_effect', true_col_name)

        #for the test set, shuffle each column and compute the different in mse, get the difference as weight
        col_names = list(train_set.columns[1:-2])
        
        #print(full_mse)
        
        for i in range(len(col_names)):
            col = col_names[i]
            test_set_shuffled = shuffle_column(test_set, col)
            pred_df_shuffled = predictTestSet(rf, test_set_shuffled)
            mse_shuffled = get_mse(pred_df_shuffled, 'rf_pred_causal_effect', true_col_name)
            #print(mse_shuffled)
            PRED_WEIGHT[i] = abs(mse_shuffled - full_mse)
            #print(PRED_WEIGHT)
    
    ret_df = pd.DataFrame()
    ret_df['Column'] = col_names
    ret_df['Weight'] = PRED_WEIGHT
    
    return ret_df

In [52]:
data = pd.read_csv("simulation//simulation_study5_large_sample_unif//large_unif.csv")
data['true_val'] = data['x1']/2
#take a subset of data
data1 = data.loc[0:200,:]
data2 = data.loc[4000:4200,:]
data = data1.append(data2)

# random forest test
#get the column names of predictors
p_str = ['x1', 'x2','x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10']
#get the column name of response
r_str = ['true_val']
#get the column name of treatment
t_str = ['treatment']

rf_train_set, rf_test_set = causal_train_test_split(data, predictors = p_str, response = r_str, treatment = t_str)
#rf = causalRandomForest(rf_train_set, max_depth = 3, min_size = 10, 
#                        criterion = 'causal', sample_ratio = 0.8, n_trees = 10)

In [49]:
weight_df = get_weighted_rf(rf_train_set, rf_test_set, n_iter = 2, true_col_name = 'true_val', 
                max_depth = 3, min_size = 10, criterion ='causal', sample_ratio = 0.8, n_trees = 10)

1.3758071035793806
1.3758071035793806
1.3711705697991527
1.3758071035793806
1.38212889362404
1.3577521289371908
1.3758071035793806
1.3758071035793806
1.374306601957993
1.3720363199260994
1.3758071035793806
1.36042036436754
1.397060391721732
1.36042036436754
1.3600833286732434
1.3552094191574473
1.36042036436754
1.36042036436754
1.3590903678019168
1.3455018145728264
1.36042036436754
1.36042036436754


In [50]:
weight_df

,Column,Weight
0,x1,0.036640
1,x2,0.000000
2,x3,0.000337
3,x4,0.005211
4,x5,0.000000
5,x6,0.000000
6,x7,0.001330
7,x8,0.014919
8,x9,0.000000
9,x10,0.000000
